In [1]:
def Runmodel_Getcriticalvolume(minerals, modal_mineralogy, csds_means, csds_stds, deviation):
    lower_bound =  modal_mineralogy-(deviation*modal_mineralogy)
    upper_bound =  modal_mineralogy+(deviation*modal_mineralogy)
    counter = 0
    highest_false = 0
    lowest_true = 1e9
    par_rock_new = (lowest_true + highest_false)/2
    while counter != 17: 
        model = SedGen(minerals, par_rock_new, modal_mineralogy, csds_means, csds_stds, learning_rate = 10000, discretization_init = False)
        proposition1 = model.simulated_volume/par_rock_new >= lower_bound
        proposition2 = model.simulated_volume/par_rock_new <= upper_bound
        if np.all(proposition1 == True) and np.all(proposition2 == True):
            lowest_true = par_rock_new
            par_rock_new = (highest_false + lowest_true)/2
            counter += 1
        else: 
            highest_false = par_rock_new
            par_rock_new = (highest_false + lowest_true)/2
            counter += 1
    print(par_rock_new)
    return par_rock_new

In [2]:
import numpy as np
import numba as nb
from numba.typed import List
import pandas as pd
import itertools
from operator import itemgetter
import gc

from scipy.stats import norm, lognorm, truncnorm
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter, deque
from sys import getsizeof

In [3]:
from sedgen.initialization import SedGen
from sedgen import initialization as ini
from sedgen import general as gen
from sedgen import preprocessing


In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time 
# before executing the Python code typed.
%autoreload 2

In [6]:
minerals = ["Q", "P", "K", "B", "O", "A"]
modal_mineralogy = np.array([0.30591989, 0.38159713, 0.26209888, 0.0188256 , 0.00799247, 0.02356603])
csds_CA_NS_means = np.array([0.309, 0.330, 0.244, 0.223, 0.120, 0.122])
csds_CA_NS_stds = np.array([0.823, 0.683, 0.817, 0.819, 0.554, 0.782])

In [7]:
Runmodel_Getcriticalvolume(minerals, modal_mineralogy, csds_CA_NS_means, csds_CA_NS_stds, deviation = 0.05)


---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A|
Initializing interfaces... |Q|P|K|B|O|A|[0 0 0 ... 0 0 0]
[1 2 1 ... 0 0 0]

Counting interfaces...
Correcting interface arrays for consistency...
too much Q 13
too few P -11
too few K -3
all good B 0
all good O 0
too much A 1
Initializing crystal size array... |Q|P|K|B|O|A|
Initializing inter-crystal breakage probability arrays...
Initializing model evolution arrays...

---SedGen model initialization finished succesfully---
---SedGen model initialization started---

Initializing modal mineralogy...
Initializing csds...
Initializing bins...
Simulating mineral occurences... |Q|P|K|B|O|A|
Initializing interfaces... |Q|P|K|B|O|A|[0 0 0 ... 0 0 0]
[1 1 2 ... 0 0 0]

Counting interfaces...
Correcting interface arrays for consistency...
too much Q 3
too few P -1
too few K -1
all good B 0
too few O -1
all good A 0
Initializing cry

24440765.380859375